# Setup

We first need to import the necessary libraries, access the data, and make a quick plot to ensure we will be analyzing the right thing.

## Import necessary libraries.

In [1]:
# Standard libraries
import os, io, glob
import datetime as dt
from pathlib import Path

# Data manipulation libraries
import numpy as np
import pandas as pd
import xarray as xr

# Data retrieval libraries
from urllib.request import urlretrieve
import requests

# Data analysis libraries
import scipy.stats as stats

# HTML parsing library
from bs4 import BeautifulSoup

# Visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# Miscellaneous
from myst_nb import glue  # used for figure numbering when exporting to LaTeX

## Set up Plotting
Then we'll establish some basic plotting rules for this notebook to keep everything looking uniform.

In [2]:

## Set up Plotting
plt.rcParams['figure.figsize'] = [10, 4]  # Set a default figure size for the notebook
plt.rcParams['figure.dpi'] = 100  # Set default resolution for inline figures

# Set the default font size for axes labels, titles and ticks
plt.rcParams['axes.titlesize'] = 16  # Set the font size for axes titles
plt.rcParams['axes.labelsize'] = 14  # Set the font size for x and y labels
plt.rcParams['xtick.labelsize'] = 12 # Set the font size for x-axis tick labels
plt.rcParams['ytick.labelsize'] = 12 # Set the font size for y-axis tick labels
plt.rcParams['font.size'] = 14 # Set the font size for the text in the figure (can affect legend)

## Define some functions
These will come up across different notebooks.

In [3]:
def process_trend_with_nan(sea_level_anomaly,weighted=False):
    # Flatten the data and get a time index
    # first ensure time is the first dimension regardless of other dimensions
    sea_level_anomaly = sea_level_anomaly.transpose('time', ...)
    sla_flat = sea_level_anomaly.values.reshape(sea_level_anomaly.shape[0], -1)
    time_index = pd.to_datetime(sea_level_anomaly.time.values).to_julian_date()

    detrended_flat = np.full_like(sla_flat, fill_value=np.nan)

    # Loop over each grid point
    for i in range(sla_flat.shape[1]):
        # Get the time series for this grid point
        y = sla_flat[:,i]
        mask = ~np.isnan(y)

        if np.any(mask):
            time_masked = time_index[mask]
            y_masked = y[mask]

            slope, intercept, _, _, _ = stats.linregress(time_masked, y_masked)
            trend = slope * time_index + intercept

            detrended_flat[:,i] = y - trend

    detrended = detrended_flat.reshape(sea_level_anomaly.shape)

    # Calculate trend magnitude
    sea_level_trend = sea_level_anomaly - detrended
    trend_mag = sea_level_trend[-1] - sea_level_trend[0]

    times = pd.to_datetime(sea_level_anomaly['time'].values)
    time_mag = (times[-1] - times[0]).days/365.25 # in years

    trend_rate = trend_mag / time_mag

    if weighted==True:
        # Calculate the weights
        weights = np.cos(np.deg2rad(sea_level_anomaly.latitude))
        weights.name = 'weights'

        # Calculate the weighted mean
        trend_mag = (trend_mag * weights).mean()
        trend_rate = (trend_rate * weights).mean()
        sea_level_trend = (sea_level_trend * weights).mean(dim=['latitude', 'longitude'])



    return trend_mag, sea_level_trend, trend_rate  